<a href="https://colab.research.google.com/github/Oussemagu/finetuning_name_variants/blob/prompt-engineering_mistral/gen_prompt_names_demi_eval_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#model


In [1]:
!pip install mistral_inference transformers sentencepiece accelerate bitsandbytes python-dotenv huggingface_hub langchain-community langchain


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━

In [4]:
!pip install python-Levenshtein langchain-huggingface pydantic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 28.1 MB/s eta 0:00:00


In [5]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `gen_names` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `gen_name

In [6]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
import torch

from langchain.llms import HuggingFacePipeline

In [7]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3",
    load_in_8bit=True,  # Enable 8-bit quantization
    device_map="auto",)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
# Use a pipeline as a high-level helper
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens=512,
                temperature=0.7,
                do_sample=True)


Device set to use cuda:0


In [9]:
llm = HuggingFacePipeline(pipeline=pipe)

In [10]:
from pydantic import BaseModel, Field
from typing import List

class OrthographicVariations(BaseModel):
    Latin: List[str] = Field(description="Three distinct Latin script variations", min_items=3, max_items=3)
    Cyrillic: List[str] = Field(description="Three distinct Cyrillic script variations", min_items=3, max_items=3)
    Arabic: List[str] = Field(description="Three distinct Arabic script variations", min_items=3, max_items=3)

class NameTransliterationOutput(BaseModel):
    Orthographic_Variations: OrthographicVariations

In [12]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser
from langchain_core.output_parsers import JsonOutputParser
from langchain.output_parsers import PydanticOutputParser # Updated import

output_parser = PydanticOutputParser(pydantic_object=NameTransliterationOutput)
# Define the prompt template
prompt_template = PromptTemplate(
    input_variables=["name"],
    template="""
[INST] You are a multilingual name transliteration expert. Your task is to generate orthographic variations for a user-provided name {name}.

User input name: {name}

Instructions:
1. Generate exactly three orthographic variations of the input name for each of the following scripts:
   - Latin
   - Cyrillic
   - Arabic
2. Ensure variations are plausible, culturally appropriate, and consistent with the input name.
3. If the input name is ambiguous (e.g., a single name like "Said"), make reasonable assumptions about its origin or context to produce accurate transliterations.
4. Do not invent names or scripts not requested.
5. Output the result as a JSON object matching this schema:
{output_parser.get_format_instructions}
6. Do not include any text outside the JSON object.

Examples:
1. User input name: Aleksandr Petrov
```json
   {{
  "Orthographic_Variations": {{
    "Latin": ["Alexander Petrov", "Alekzandr Petroff", "Alyaksandr Petrou"],
    "Cyrillic": ["Александр Петров", "Александар Пятров", "Аляксандр Пятроў"],
    "Arabic": ["ألكسندر بيتروف", "أليكسندر بيتروف", "أليكسندار بيتروف"]
  }}
}}


2. User input name: فاطمة الزهراء
   {{
  "Orthographic_Variations": {{
    "Latin": ["Fatima Al-Zahra", "Faatimah Az-Zahraa", "Fatma Zahraa"],
    "Cyrillic": ["Фатима Аз-Захра", "Фатма Аль-Захраа", "Фатыма Аз-Захра"],
    "Arabic": ["فاطمة الزهراء", "فاطمه الزهراء", "فاطما الزهراء"]
  }}
}}

3. User input name: Саид Петр
   {{
  "Orthographic_Variations": {{
    "Latin": ["Said Petr", "Sayed Pyotr", "Sayid Petar"],
    "Cyrillic": ["Саид Пётр", "Сайд Петар", "Сайид Пьотр"],
    "Arabic": ["سعيد بطرس", "سيد بطرس", "سعيّد بطرس"]
  }}
}}
[/INST]
""",
    partial_variables={"output_parser": output_parser}
)


# Test names
test_names = ["sawsen bent foulenn"]

# Run the prompt for each test name
for name in test_names:
    formatted_prompt = prompt_template.format(name=name, output_parser=output_parser)
    messages = [
        {"role": "user", "content": formatted_prompt}
    ]
    print(f"\nInput Name: {name}")
    print("Output:")
    print(llm.invoke(messages))  # Use invoke for cleaner output

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Input Name: sawsen bent foulenn
Output:
Human: 
[INST] You are a multilingual name transliteration expert. Your task is to generate orthographic variations for a user-provided name.

User input name: sawsen bent foulenn

Instructions:
1. Generate exactly three orthographic variations of the input name for each of the following scripts:
   - Latin
   - Cyrillic
   - Arabic
2. Ensure variations are plausible, culturally appropriate, and consistent with the input name.
3. If the input name is ambiguous (e.g., a single name like "Said"), make reasonable assumptions about its origin or context to produce accurate transliterations.
4. Do not invent names or scripts not requested.
5. Output the result as a JSON object matching this schema:
<bound method PydanticOutputParser.get_format_instructions of PydanticOutputParser(pydantic_object=<class '__main__.NameTransliterationOutput'>)>
6. Do not include any text outside the JSON object.

Examples:
1. User input name: Aleksandr Petrov
```json
  

#evaluation


In [14]:
!pip install sacrebleu

In [33]:
# Install dependencies
!pip install python-Levenshtein pandas ragas nltk --quiet



# Import other modules
import json
import pandas as pd
from Levenshtein import distance
from langchain_core.exceptions import OutputParserException
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from ragas.metrics import BleuScore, AnswerSimilarity
from ragas.dataset_schema import SingleTurnSample
from ragas.metrics import SemanticSimilarity
from ragas.embeddings import LangchainEmbeddingsWrapper
import itertools

In [34]:
import nltk
nltk.download('punkt', quiet=True)

True

In [39]:
# Test dataset (10 entries)
test_dataset = [
    {
        "input": "mohamed naji dridi",
        "ground_truth": {
            "Latin": ["Mohamed Naji Dridi", "Mohammed Nagy Dridi", "Muhammad Nagee Dridi"],
            "Cyrillic": ["Мохамед Нажи Дриди", "Мохаммед Наги Дриди", "Мухаммад Нагий Дриди"],
            "Arabic": ["محمد ناجي دريدي", "محمّد ناجي دريدي", "محمد ناجى دريدى"]
        }
    },
    {
        "input": "Fatima",
        "ground_truth": {
            "Latin": ["Fatima", "Faatimah", "Fatma"],
            "Cyrillic": ["Фатима", "Фатимах", "Фатма"],
            "Arabic": ["فاطمة", "فاطمه", "فاطما"]
        }
    },
    {
        "input": "Aleksandr Ivanov",
        "ground_truth": {
            "Latin": ["Alexander Ivanov", "Alekzandr Ivanow", "Alexandr Ivanow"],
            "Cyrillic": ["Александр Иванов", "Александар Иваноф", "Аляксандр Іванов"],
            "Arabic": ["ألكسندر إيفانوف", "أليكسندر إيفانوف", "أليكسندار إيفانوف"]
        }
    },
    {
        "input": "عمر بن الخطاب",
        "ground_truth": {
            "Latin": ["Omar ibn al-Khattab", "Umar bin al-Khattab", "Omer ben al-Khatab"],
            "Cyrillic": ["Омар ибн аль-Хаттаб", "Умар бин аль-Хаттаб", "Омер бен аль-Хатаб"],
            "Arabic": ["عمر بن الخطاب", "عمر بن الخطّاب", "عُمر بن الخطاب"]
        }
    },
    {
        "input": "Sofia Petrova",
        "ground_truth": {
            "Latin": ["Sofia Petrova", "Sophia Petrova", "Sofiya Petrovna"],
            "Cyrillic": ["София Петрова", "Софья Петрова", "Сафія Пятрова"],
            "Arabic": ["صوفيا بيتروفا", "صوفية بيتروفا", "سفيا بيتروفا"]
        }
    },
    {
        "input": "حسن علي",
        "ground_truth": {
            "Latin": ["Hassan Ali", "Hasan Aly", "Hussain Ali"],
            "Cyrillic": ["Хассан Али", "Хасан Алы", "Хусейн Али"],
            "Arabic": ["حسن علي", "حسّن علي", "حسن عليّ"]
        }
    },
    {
        "input": "Maria Gonzalez",
        "ground_truth": {
            "Latin": ["Maria Gonzalez", "Mariya Gonzales", "Marie Gonzalvez"],
            "Cyrillic": ["Мария Гонсалес", "Мария Гонсалез", "Мари Гонсалвес"],
            "Arabic": ["ماريا غونزاليس", "مريا غونزاليز", "ماري غونزالفيس"]
        }
    },
    {
        "input": "خالد محمود",
        "ground_truth": {
            "Latin": ["Khaled Mahmoud", "Khalid Mahmud", "Khalid Mahmod"],
            "Cyrillic": ["Халед Махмуд", "Халид Махмуд", "Халед Махмод"],
            "Arabic": ["خالد محمود", "خالد محمّود", "خالد مَحمود"]
        }
    },
    {
        "input": "Said",
        "ground_truth": {
            "Latin": ["Said", "Sayed", "Sayid"],
            "Cyrillic": ["Саид", "Сайед", "Сайид"],
            "Arabic": ["سعيد", "سيد", "سعيّد"]
        }
    },
    {
        "input": "Anna Kowalski",
        "ground_truth": {
            "Latin": ["Anna Kowalski", "Anya Kowalsky", "Hanna Kowalska"],
            "Cyrillic": ["Анна Ковальски", "Аня Ковальский", "Ханна Ковальска"],
            "Arabic": ["آنا كوالسكي", "أنيا كوالسكي", "هنا كوالسكا"]
        }
    }
]

# Save dataset
with open("test_dataset.json", "w", encoding="utf-8") as f:
    json.dump(test_dataset, f, ensure_ascii=False, indent=2)

In [50]:
import pandas as pd
from Levenshtein import distance
from langchain_core.exceptions import OutputParserException
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import itertools
from sentence_transformers import SentenceTransformer, util

# Initialize metrics
st_model = SentenceTransformer('all-MiniLM-L6-v2')
smoothie = SmoothingFunction().method4

def evaluate_prompt(llm, prompt_template, test_dataset, output_parser):
    results = []

    for test_case in test_dataset:
        input_name = test_case["input"]
        ground_truth = test_case["ground_truth"]

        # Generate output
        formatted_prompt = prompt_template.format(name=input_name)
        messages = [{"role": "user", "content": formatted_prompt}]
        raw_output = llm.invoke(messages)
        print(f"Input: {input_name}, Raw output: {raw_output}")  # Debug

        # Parse JSON output
        try:
            json_start = raw_output.find("{")
            json_end = raw_output.rfind("}") + 1
            if json_start == -1 or json_end == -1:
                raise ValueError("No JSON found in output")
            json_str = raw_output[json_start:json_end]
            parsed_output = output_parser.parse(json_str)
            variations = parsed_output.Orthographic_Variations
            latin = variations.Latin
            cyrillic = variations.Cyrillic
            arabic = variations.Arabic

            format_score = 1 if (len(latin) == 3 and len(cyrillic) == 3 and len(arabic) == 3) else 0
        except (ValueError, OutputParserException) as e:
            print(f"Parsing error for {input_name}: {e}")
            latin, cyrillic, arabic = [], [], []
            format_score = 0

        # Tokenize for BLEU/Self-BLEU (character-level)
        def tokenize(text):
            return list(text)  # Character-level for names

        # Levenshtein scores
        levenshtein_scores = {
            "Latin": [min(distance(gen, gt) / max(len(gen), len(gt), 1) for gt in ground_truth["Latin"]) for gen in latin] or [1.0],
            "Cyrillic": [min(distance(gen, gt) / max(len(gen), len(gt), 1) for gt in ground_truth["Cyrillic"]) for gen in cyrillic] or [1.0],
            "Arabic": [min(distance(gen, gt) / max(len(gen), len(gt), 1) for gt in ground_truth["Arabic"]) for gen in arabic] or [1.0]
        }

        # NLTK BLEU scores
        bleu_scores = {"Latin": [], "Cyrillic": [], "Arabic": []}
        for gen, gt_list in [(latin, ground_truth["Latin"]), (cyrillic, ground_truth["Cyrillic"]), (arabic, ground_truth["Arabic"])]:
            script = "Latin" if gt_list is ground_truth["Latin"] else "Cyrillic" if gt_list is ground_truth["Cyrillic"] else "Arabic"
            for g in gen:
                gen_tokens = tokenize(g)
                gt_tokens_list = [tokenize(gt) for gt in gt_list]
                bleu_score = sentence_bleu(
                    gt_tokens_list,
                    gen_tokens,
                    weights=(0.5, 0.5, 0, 0),
                    smoothing_function=smoothie
                )
                bleu_scores[script].append(bleu_score)

        # Sentence-transformers semantic similarity
        semantic_scores = {"Latin": [], "Cyrillic": [], "Arabic": []}
        for gen, gt_list in [(latin, ground_truth["Latin"]), (cyrillic, ground_truth["Cyrillic"]), (arabic, ground_truth["Arabic"])]:
            script = "Latin" if gt_list is ground_truth["Latin"] else "Cyrillic" if gt_list is ground_truth["Cyrillic"] else "Arabic"
            for g in gen:
                g_emb = st_model.encode(g, convert_to_tensor=True)
                gt_embs = [st_model.encode(gt, convert_to_tensor=True) for gt in gt_list]
                sem_score = max(float(util.cos_sim(g_emb, gt_emb)) for gt_emb in gt_embs)
                semantic_scores[script].append(sem_score)

        # Self-BLEU
        self_bleu_scores = {"Latin": [], "Cyrillic": [], "Arabic": []}
        for gen_list, script in [(latin, "Latin"), (cyrillic, "Cyrillic"), (arabic, "Arabic")]:
            if len(gen_list) >= 2:
                pairs = list(itertools.combinations(gen_list, 2))
                for g1, g2 in pairs:
                    tokens1, tokens2 = tokenize(g1), tokenize(g2)
                    score = sentence_bleu([tokens1], tokens2, weights=(0.5, 0.5, 0, 0), smoothing_function=smoothie)
                    self_bleu_scores[script].append(score)
            else:
                self_bleu_scores[script].append(1.0)  # Penalty for insufficient variations

        avg_metrics = {
            "Levenshtein_Latin": sum(levenshtein_scores["Latin"]) / max(len(levenshtein_scores["Latin"]), 1),
            "Levenshtein_Cyrillic": sum(levenshtein_scores["Cyrillic"]) / max(len(levenshtein_scores["Cyrillic"]), 1),
            "Levenshtein_Arabic": sum(levenshtein_scores["Arabic"]) / max(len(levenshtein_scores["Arabic"]), 1),
            "BLEU_Latin": sum(bleu_scores["Latin"]) / max(len(bleu_scores["Latin"]), 1),
            "BLEU_Cyrillic": sum(bleu_scores["Cyrillic"]) / max(len(bleu_scores["Cyrillic"]), 1),
            "BLEU_Arabic": sum(bleu_scores["Arabic"]) / max(len(bleu_scores["Arabic"]), 1),
            "Semantic_Latin": sum(semantic_scores["Latin"]) / max(len(semantic_scores["Latin"]), 1),
            "Semantic_Cyrillic": sum(semantic_scores["Cyrillic"]) / max(len(semantic_scores["Cyrillic"]), 1),
            "Semantic_Arabic": sum(semantic_scores["Arabic"]) / max(len(semantic_scores["Arabic"]), 1),
            "Self_BLEU_Latin": sum(self_bleu_scores["Latin"]) / max(len(self_bleu_scores["Latin"]), 1) if self_bleu_scores["Latin"] else 1.0,
            "Self_BLEU_Cyrillic": sum(self_bleu_scores["Cyrillic"]) / max(len(self_bleu_scores["Cyrillic"]), 1) if self_bleu_scores["Cyrillic"] else 1.0,
            "Self_BLEU_Arabic": sum(self_bleu_scores["Arabic"]) / max(len(self_bleu_scores["Arabic"]), 1) if self_bleu_scores["Arabic"] else 1.0
        }

        results.append({
            "Input": input_name,
            "Generated_Latin": ", ".join(latin),
            "Generated_Cyrillic": ", ".join(cyrillic),
            "Generated_Arabic": ", ".join(arabic),
            "Levenshtein_Latin": avg_metrics["Levenshtein_Latin"],
            "Levenshtein_Cyrillic": avg_metrics["Levenshtein_Cyrillic"],
            "Levenshtein_Arabic": avg_metrics["Levenshtein_Arabic"],
            "BLEU_Latin": avg_metrics["BLEU_Latin"],
            "BLEU_Cyrillic": avg_metrics["BLEU_Cyrillic"],
            "BLEU_Arabic": avg_metrics["BLEU_Arabic"],
            "Semantic_Latin": avg_metrics["Semantic_Latin"],
            "Semantic_Cyrillic": avg_metrics["Semantic_Cyrillic"],
            "Semantic_Arabic": avg_metrics["Semantic_Arabic"],
            "Self_BLEU_Latin": avg_metrics["Self_BLEU_Latin"],
            "Self_BLEU_Cyrillic": avg_metrics["Self_BLEU_Cyrillic"],
            "Self_BLEU_Arabic": avg_metrics["Self_BLEU_Arabic"],
            "Format_Correct": format_score
        })

    df = pd.DataFrame(results)
    avg_metrics = {
        "Avg_Levenshtein_Latin": df["Levenshtein_Latin"].mean(),
        "Avg_Levenshtein_Cyrillic": df["Levenshtein_Cyrillic"].mean(),
        "Avg_Levenshtein_Arabic": df["Levenshtein_Arabic"].mean(),
        "Avg_BLEU_Latin": df["BLEU_Latin"].mean(),
        "Avg_BLEU_Cyrillic": df["BLEU_Cyrillic"].mean(),
        "Avg_BLEU_Arabic": df["BLEU_Arabic"].mean(),
        "Avg_Semantic_Latin": df["Semantic_Latin"].mean(),
        "Avg_Semantic_Cyrillic": df["Semantic_Cyrillic"].mean(),
        "Avg_Semantic_Arabic": df["Semantic_Arabic"].mean(),
        "Avg_Self_BLEU_Latin": df["Self_BLEU_Latin"].mean(),
        "Avg_Self_BLEU_Cyrillic": df["Self_BLEU_Cyrillic"].mean(),
        "Avg_Self_BLEU_Arabic": df["Self_BLEU_Arabic"].mean(),
        "Format_Accuracy": df["Format_Correct"].mean()
    }

    return df, avg_metrics


# Run evaluation
df_results, avg_metrics = evaluate_prompt(llm, prompt_template, test_dataset, output_parser)
df_results.to_csv("prompt_evaluation_no_exact_match.csv", index=False)

print("Evaluation Results:")
print(df_results)
print("\nAverage Metrics:")
for k, v in avg_metrics.items():
    print(f"{k}: {v:.2f}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Input: mohamed naji dridi, Raw output: Human: 
[INST] You are a multilingual name transliteration expert. Your task is to generate orthographic variations for a user-provided name.

User input name: mohamed naji dridi

Instructions:
1. Generate exactly three orthographic variations of the input name for each of the following scripts:
   - Latin
   - Cyrillic
   - Arabic
2. Ensure variations are plausible, culturally appropriate, and consistent with the input name.
3. If the input name is ambiguous (e.g., a single name like "Said"), make reasonable assumptions about its origin or context to produce accurate transliterations.
4. Do not invent names or scripts not requested.
5. Output the result as a JSON object matching this schema:
<bound method PydanticOutputParser.get_format_instructions of PydanticOutputParser(pydantic_object=<class '__main__.NameTransliterationOutput'>)>
6. Do not include any text outside the JSON object.

Examples:
1. User input name: Aleksandr Petrov
```json
   {


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Input: Fatima, Raw output: Human: 
[INST] You are a multilingual name transliteration expert. Your task is to generate orthographic variations for a user-provided name.

User input name: Fatima

Instructions:
1. Generate exactly three orthographic variations of the input name for each of the following scripts:
   - Latin
   - Cyrillic
   - Arabic
2. Ensure variations are plausible, culturally appropriate, and consistent with the input name.
3. If the input name is ambiguous (e.g., a single name like "Said"), make reasonable assumptions about its origin or context to produce accurate transliterations.
4. Do not invent names or scripts not requested.
5. Output the result as a JSON object matching this schema:
<bound method PydanticOutputParser.get_format_instructions of PydanticOutputParser(pydantic_object=<class '__main__.NameTransliterationOutput'>)>
6. Do not include any text outside the JSON object.

Examples:
1. User input name: Aleksandr Petrov
```json
   {
  "Orthographic_Variatio

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Input: خالد محمود, Raw output: Human: 
[INST] You are a multilingual name transliteration expert. Your task is to generate orthographic variations for a user-provided name.

User input name: خالد محمود

Instructions:
1. Generate exactly three orthographic variations of the input name for each of the following scripts:
   - Latin
   - Cyrillic
   - Arabic
2. Ensure variations are plausible, culturally appropriate, and consistent with the input name.
3. If the input name is ambiguous (e.g., a single name like "Said"), make reasonable assumptions about its origin or context to produce accurate transliterations.
4. Do not invent names or scripts not requested.
5. Output the result as a JSON object matching this schema:
<bound method PydanticOutputParser.get_format_instructions of PydanticOutputParser(pydantic_object=<class '__main__.NameTransliterationOutput'>)>
6. Do not include any text outside the JSON object.

Examples:
1. User input name: Aleksandr Petrov
```json
   {
  "Orthographic_

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Input: Said, Raw output: Human: 
[INST] You are a multilingual name transliteration expert. Your task is to generate orthographic variations for a user-provided name.

User input name: Said

Instructions:
1. Generate exactly three orthographic variations of the input name for each of the following scripts:
   - Latin
   - Cyrillic
   - Arabic
2. Ensure variations are plausible, culturally appropriate, and consistent with the input name.
3. If the input name is ambiguous (e.g., a single name like "Said"), make reasonable assumptions about its origin or context to produce accurate transliterations.
4. Do not invent names or scripts not requested.
5. Output the result as a JSON object matching this schema:
<bound method PydanticOutputParser.get_format_instructions of PydanticOutputParser(pydantic_object=<class '__main__.NameTransliterationOutput'>)>
6. Do not include any text outside the JSON object.

Examples:
1. User input name: Aleksandr Petrov
```json
   {
  "Orthographic_Variations":

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Input: Anna Kowalski, Raw output: Human: 
[INST] You are a multilingual name transliteration expert. Your task is to generate orthographic variations for a user-provided name.

User input name: Anna Kowalski

Instructions:
1. Generate exactly three orthographic variations of the input name for each of the following scripts:
   - Latin
   - Cyrillic
   - Arabic
2. Ensure variations are plausible, culturally appropriate, and consistent with the input name.
3. If the input name is ambiguous (e.g., a single name like "Said"), make reasonable assumptions about its origin or context to produce accurate transliterations.
4. Do not invent names or scripts not requested.
5. Output the result as a JSON object matching this schema:
<bound method PydanticOutputParser.get_format_instructions of PydanticOutputParser(pydantic_object=<class '__main__.NameTransliterationOutput'>)>
6. Do not include any text outside the JSON object.

Examples:
1. User input name: Aleksandr Petrov
```json
   {
  "Orthogr

In [51]:
import pandas as pd
from Levenshtein import distance
from langchain_core.exceptions import OutputParserException
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import itertools
from sentence_transformers import SentenceTransformer, util

# Initialize metrics
st_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
smoothie = SmoothingFunction().method4

def evaluate_prompt(llm, prompt_template, test_dataset, output_parser):
    results = []

    for test_case in test_dataset:
        input_name = test_case["input"]
        ground_truth = test_case["ground_truth"]

        # Generate output
        formatted_prompt = prompt_template.format(name=input_name)
        messages = [{"role": "user", "content": formatted_prompt}]
        raw_output = llm.invoke(messages)
        print(f"Input: {input_name}, Raw output: {raw_output}")  # Debug

        # Parse JSON output
        try:
            json_start = raw_output.find("{")
            json_end = raw_output.rfind("}") + 1
            if json_start == -1 or json_end == -1:
                raise ValueError("No JSON found in output")
            json_str = raw_output[json_start:json_end]
            parsed_output = output_parser.parse(json_str)
            variations = parsed_output.Orthographic_Variations
            latin = variations.Latin
            cyrillic = variations.Cyrillic
            arabic = variations.Arabic
            print(f"Parsed variations for {input_name}: Latin={latin}, Cyrillic={cyrillic}, Arabic={arabic}")  # Debug

            format_score = 1 if (len(latin) == 3 and len(cyrillic) == 3 and len(arabic) == 3) else 0
        except (ValueError, OutputParserException) as e:
            print(f"Parsing error for {input_name}: {e}")
            latin, cyrillic, arabic = [], [], []
            format_score = 0

        # Tokenize for BLEU/Self-BLEU (character-level)
        def tokenize(text):
            return list(text)  # Character-level for names

        # Levenshtein scores
        levenshtein_scores = {
            "Latin": [min(distance(gen, gt) / max(len(gen), len(gt), 1) for gt in ground_truth["Latin"]) for gen in latin] or [1.0],
            "Cyrillic": [min(distance(gen, gt) / max(len(gen), len(gt), 1) for gt in ground_truth["Cyrillic"]) for gen in cyrillic] or [1.0],
            "Arabic": [min(distance(gen, gt) / max(len(gen), len(gt), 1) for gt in ground_truth["Arabic"]) for gen in arabic] or [1.0]
        }

        # NLTK BLEU scores
        bleu_scores = {"Latin": [], "Cyrillic": [], "Arabic": []}
        for gen, gt_list in [(latin, ground_truth["Latin"]), (cyrillic, ground_truth["Cyrillic"]), (arabic, ground_truth["Arabic"])]:
            script = "Latin" if gt_list is ground_truth["Latin"] else "Cyrillic" if gt_list is ground_truth["Cyrillic"] else "Arabic"
            for g in gen:
                gen_tokens = tokenize(g)
                gt_tokens_list = [tokenize(gt) for gt in gt_list]
                bleu_score = sentence_bleu(
                    gt_tokens_list,
                    gen_tokens,
                    weights=(0.5, 0.5, 0, 0),
                    smoothing_function=smoothie
                )
                bleu_scores[script].append(bleu_score)

        # Sentence-transformers semantic similarity
        semantic_scores = {"Latin": [], "Cyrillic": [], "Arabic": []}
        for gen, gt_list in [(latin, ground_truth["Latin"]), (cyrillic, ground_truth["Cyrillic"]), (arabic, ground_truth["Arabic"])]:
            script = "Latin" if gt_list is ground_truth["Latin"] else "Cyrillic" if gt_list is ground_truth["Cyrillic"] else "Arabic"
            for g in gen:
                g_emb = st_model.encode(g, convert_to_tensor=True)
                gt_embs = [st_model.encode(gt, convert_to_tensor=True) for gt in gt_list]
                sem_score = max(float(util.cos_sim(g_emb, gt_emb)) for gt_emb in gt_embs)
                semantic_scores[script].append(sem_score)

        # Self-BLEU
        self_bleu_scores = {"Latin": [], "Cyrillic": [], "Arabic": []}
        for gen_list, script in [(latin, "Latin"), (cyrillic, "Cyrillic"), (arabic, "Arabic")]:
            if len(gen_list) >= 2:
                pairs = list(itertools.combinations(gen_list, 2))
                for g1, g2 in pairs:
                    tokens1, tokens2 = tokenize(g1), tokenize(g2)
                    score = sentence_bleu([tokens1], tokens2, weights=(0.5, 0.5, 0, 0), smoothing_function=smoothie)
                    self_bleu_scores[script].append(score)
            else:
                self_bleu_scores[script].append(1.0)  # Penalty for insufficient variations

        avg_metrics = {
            "Levenshtein_Latin": sum(levenshtein_scores["Latin"]) / max(len(levenshtein_scores["Latin"]), 1),
            "Levenshtein_Cyrillic": sum(levenshtein_scores["Cyrillic"]) / max(len(levenshtein_scores["Cyrillic"]), 1),
            "Levenshtein_Arabic": sum(levenshtein_scores["Arabic"]) / max(len(levenshtein_scores["Arabic"]), 1),
            "BLEU_Latin": sum(bleu_scores["Latin"]) / max(len(bleu_scores["Latin"]), 1),
            "BLEU_Cyrillic": sum(bleu_scores["Cyrillic"]) / max(len(bleu_scores["Cyrillic"]), 1),
            "BLEU_Arabic": sum(bleu_scores["Arabic"]) / max(len(bleu_scores["Arabic"]), 1),
            "Semantic_Latin": sum(semantic_scores["Latin"]) / max(len(semantic_scores["Latin"]), 1),
            "Semantic_Cyrillic": sum(semantic_scores["Cyrillic"]) / max(len(semantic_scores["Cyrillic"]), 1),
            "Semantic_Arabic": sum(semantic_scores["Arabic"]) / max(len(semantic_scores["Arabic"]), 1),
            "Self_BLEU_Latin": sum(self_bleu_scores["Latin"]) / max(len(self_bleu_scores["Latin"]), 1) if self_bleu_scores["Latin"] else 1.0,
            "Self_BLEU_Cyrillic": sum(self_bleu_scores["Cyrillic"]) / max(len(self_bleu_scores["Cyrillic"]), 1) if self_bleu_scores["Cyrillic"] else 1.0,
            "Self_BLEU_Arabic": sum(self_bleu_scores["Arabic"]) / max(len(self_bleu_scores["Arabic"]), 1) if self_bleu_scores["Arabic"] else 1.0
        }

        results.append({
            "Input": input_name,
            "Generated_Latin": ", ".join(latin),
            "Generated_Cyrillic": ", ".join(cyrillic),
            "Generated_Arabic": ", ".join(arabic),
            "Levenshtein_Latin": avg_metrics["Levenshtein_Latin"],
            "Levenshtein_Cyrillic": avg_metrics["Levenshtein_Cyrillic"],
            "Levenshtein_Arabic": avg_metrics["Levenshtein_Arabic"],
            "BLEU_Latin": avg_metrics["BLEU_Latin"],
            "BLEU_Cyrillic": avg_metrics["BLEU_Cyrillic"],
            "BLEU_Arabic": avg_metrics["BLEU_Arabic"],
            "Semantic_Latin": avg_metrics["Semantic_Latin"],
            "Semantic_Cyrillic": avg_metrics["Semantic_Cyrillic"],
            "Semantic_Arabic": avg_metrics["Semantic_Arabic"],
            "Self_BLEU_Latin": avg_metrics["Self_BLEU_Latin"],
            "Self_BLEU_Cyrillic": avg_metrics["Self_BLEU_Cyrillic"],
            "Self_BLEU_Arabic": avg_metrics["Self_BLEU_Arabic"],
            "Format_Correct": format_score
        })

    df = pd.DataFrame(results)
    avg_metrics = {
        "Avg_Levenshtein_Latin": df["Levenshtein_Latin"].mean(),
        "Avg_Levenshtein_Cyrillic": df["Levenshtein_Cyrillic"].mean(),
        "Avg_Levenshtein_Arabic": df["Levenshtein_Arabic"].mean(),
        "Avg_BLEU_Latin": df["BLEU_Latin"].mean(),
        "Avg_BLEU_Cyrillic": df["BLEU_Cyrillic"].mean(),
        "Avg_BLEU_Arabic": df["BLEU_Arabic"].mean(),
        "Avg_Semantic_Latin": df["Semantic_Latin"].mean(),
        "Avg_Semantic_Cyrillic": df["Semantic_Cyrillic"].mean(),
        "Avg_Semantic_Arabic": df["Semantic_Arabic"].mean(),
        "Avg_Self_BLEU_Latin": df["Self_BLEU_Latin"].mean(),
        "Avg_Self_BLEU_Cyrillic": df["Self_BLEU_Cyrillic"].mean(),
        "Avg_Self_BLEU_Arabic": df["Self_BLEU_Arabic"].mean(),
        "Format_Accuracy": df["Format_Correct"].mean()
    }

    return df, avg_metrics

# Install dependencies
!pip install python-Levenshtein pandas nltk sentence-transformers --quiet

# Run evaluation
df_results, avg_metrics = evaluate_prompt(llm, prompt_template, test_dataset, output_parser)
df_results.to_csv("prompt_evaluation_no_exact_match.csv", index=False)

print("Evaluation Results:")
print(df_results)
print("\nAverage Metrics:")
for k, v in avg_metrics.items():
    print(f"{k}: {v:.2f}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Input: mohamed naji dridi, Raw output: Human: 
[INST] You are a multilingual name transliteration expert. Your task is to generate orthographic variations for a user-provided name.

User input name: mohamed naji dridi

Instructions:
1. Generate exactly three orthographic variations of the input name for each of the following scripts:
   - Latin
   - Cyrillic
   - Arabic
2. Ensure variations are plausible, culturally appropriate, and consistent with the input name.
3. If the input name is ambiguous (e.g., a single name like "Said"), make reasonable assumptions about its origin or context to produce accurate transliterations.
4. Do not invent names or scripts not requested.
5. Output the result as a JSON object matching this schema:
<bound method PydanticOutputParser.get_format_instructions of PydanticOutputParser(pydantic_object=<class '__main__.NameTransliterationOutput'>)>
6. Do not include any text outside the JSON object.

Examples:
1. User input name: Aleksandr Petrov
```json
   {


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Input: Fatima, Raw output: Human: 
[INST] You are a multilingual name transliteration expert. Your task is to generate orthographic variations for a user-provided name.

User input name: Fatima

Instructions:
1. Generate exactly three orthographic variations of the input name for each of the following scripts:
   - Latin
   - Cyrillic
   - Arabic
2. Ensure variations are plausible, culturally appropriate, and consistent with the input name.
3. If the input name is ambiguous (e.g., a single name like "Said"), make reasonable assumptions about its origin or context to produce accurate transliterations.
4. Do not invent names or scripts not requested.
5. Output the result as a JSON object matching this schema:
<bound method PydanticOutputParser.get_format_instructions of PydanticOutputParser(pydantic_object=<class '__main__.NameTransliterationOutput'>)>
6. Do not include any text outside the JSON object.

Examples:
1. User input name: Aleksandr Petrov
```json
   {
  "Orthographic_Variatio

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Input: خالد محمود, Raw output: Human: 
[INST] You are a multilingual name transliteration expert. Your task is to generate orthographic variations for a user-provided name.

User input name: خالد محمود

Instructions:
1. Generate exactly three orthographic variations of the input name for each of the following scripts:
   - Latin
   - Cyrillic
   - Arabic
2. Ensure variations are plausible, culturally appropriate, and consistent with the input name.
3. If the input name is ambiguous (e.g., a single name like "Said"), make reasonable assumptions about its origin or context to produce accurate transliterations.
4. Do not invent names or scripts not requested.
5. Output the result as a JSON object matching this schema:
<bound method PydanticOutputParser.get_format_instructions of PydanticOutputParser(pydantic_object=<class '__main__.NameTransliterationOutput'>)>
6. Do not include any text outside the JSON object.

Examples:
1. User input name: Aleksandr Petrov
```json
   {
  "Orthographic_

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Input: Said, Raw output: Human: 
[INST] You are a multilingual name transliteration expert. Your task is to generate orthographic variations for a user-provided name.

User input name: Said

Instructions:
1. Generate exactly three orthographic variations of the input name for each of the following scripts:
   - Latin
   - Cyrillic
   - Arabic
2. Ensure variations are plausible, culturally appropriate, and consistent with the input name.
3. If the input name is ambiguous (e.g., a single name like "Said"), make reasonable assumptions about its origin or context to produce accurate transliterations.
4. Do not invent names or scripts not requested.
5. Output the result as a JSON object matching this schema:
<bound method PydanticOutputParser.get_format_instructions of PydanticOutputParser(pydantic_object=<class '__main__.NameTransliterationOutput'>)>
6. Do not include any text outside the JSON object.

Examples:
1. User input name: Aleksandr Petrov
```json
   {
  "Orthographic_Variations":

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Input: Anna Kowalski, Raw output: Human: 
[INST] You are a multilingual name transliteration expert. Your task is to generate orthographic variations for a user-provided name.

User input name: Anna Kowalski

Instructions:
1. Generate exactly three orthographic variations of the input name for each of the following scripts:
   - Latin
   - Cyrillic
   - Arabic
2. Ensure variations are plausible, culturally appropriate, and consistent with the input name.
3. If the input name is ambiguous (e.g., a single name like "Said"), make reasonable assumptions about its origin or context to produce accurate transliterations.
4. Do not invent names or scripts not requested.
5. Output the result as a JSON object matching this schema:
<bound method PydanticOutputParser.get_format_instructions of PydanticOutputParser(pydantic_object=<class '__main__.NameTransliterationOutput'>)>
6. Do not include any text outside the JSON object.

Examples:
1. User input name: Aleksandr Petrov
```json
   {
  "Orthogr